# **Layer 1: Baseline DeiT environment**

DeiT’s baseline training script expects a teacher model name and distillation settings via CLI flags in main.py (e.g., --teacher-model, --teacher-path, --distillation-type).
GitHub
+1

So the “base environment” Layer 1 must include:

DeiT repo (cloned)

PyTorch (Colab default) + GPU

timm installed (for both student and teacher models)

compatibility patches if any (because Colab uses new torch/timm)

Install PyTorch without pinning

In [ ]:
!pip -q install --upgrade pip
!pip -q install torch torchvision torchaudio

Verify

In [ ]:
import torch
print(torch.__version__)
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


Clone the baseline repo (official DeiT)

In [ ]:
%cd /content
!git clone https://github.com/facebookresearch/deit.git
%cd /content/deit
!grep -n "torch" -n requirements.txt || true

/content
fatal: destination path 'deit' already exists and is not an empty directory.
/content/deit


Colab Compatibility Fixes

1. torch pin removal

2. timm API changes

3. kwargs popping (pretrained_cfg, cache_dir, etc.)



Patch requirements.txt to remove torch pins

In [ ]:
%cd /content/deit

!python - << 'PY'
from pathlib import Path
p = Path("requirements.txt")
lines = p.read_text().splitlines()

filtered = []
removed = []
for line in lines:
    s = line.strip()
    if s.startswith("torch==") or s.startswith("torchvision==") or s.startswith("torchaudio=="):
        removed.append(line)
        continue
    filtered.append(line)

p.write_text("\n".join(filtered) + "\n")
print("✅ Removed these pinned lines:")
for r in removed:
    print("  -", r)

/content/deit
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ Removed these pinned lines:


Verify Pins are gone!i.e torch==1.13.1 pin was removed

In [ ]:
!grep -nE "torch|torchvision|torchaudio" requirements.txt || echo "✅ No torch pins remain"

✅ No torch pins remain


Install the baseline dependencies

In [ ]:
pip install "jedi>=0.16,<0.19"

In [ ]:
!pip -q uninstall -y timm
!pip -q install "jedi>=0.16,<0.19"
# !pip -q install timm==0.6.13 submitit
!pip -q install timm==0.4.12 submitit


In [ ]:
#!pip -q uninstall -y timm
#!pip -q install -U pip setuptools wheel
#!pip -q install -U "timm>=1.0.0"

Verify

In [ ]:
!python -c "import timm; print('timm:', timm.__version__)"
#0.4.12

timm: 0.4.12


**Restart the Session**

In [ ]:
!python - << 'PY'
from pathlib import Path

p = Path("/usr/local/lib/python3.12/dist-packages/timm/data/__init__.py")
txt = p.read_text()

needle = "OPENAI_CLIP_MEAN"
if needle in txt:
    print("✅ timm.data already mentions OPENAI_CLIP_MEAN; no patch needed.")
else:
    patch = """

# --- Colab patch: expose CLIP normalization constants for older exports ---
try:
    from .constants import OPENAI_CLIP_MEAN, OPENAI_CLIP_STD  # timm versions where defined in constants
except Exception:
    # Standard OpenAI CLIP normalization
    OPENAI_CLIP_MEAN = (0.48145466, 0.4578275, 0.40821073)
    OPENAI_CLIP_STD  = (0.26862954, 0.26130258, 0.27577711)
# --- end patch ---
"""
    p.write_text(txt + patch)
    print("✅ Patched:", p)

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ Patched: /usr/local/lib/python3.12/dist-packages/timm/data/__init__.py


Runtime → Restart runtime

In [ ]:
#!pip -q install timm submitit

In [ ]:
%cd /content/deit
from models import deit_tiny_patch16_224
m = deit_tiny_patch16_224()
print("✅ DeiT model instantiated successfully")

/content/deit
✅ DeiT model instantiated successfully


In [ ]:
import torch, timm
print(torch.__version__)
print(timm.__version__)
print(torch.cuda.is_available())

2.9.0+cu126
0.4.12
True


Download Tiny-ImageNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content
!wget -q http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

/content
replace tiny-imagenet-200/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


Fix Tiny-ImageNet validation folder

In [ ]:
!python - << 'EOF'
import shutil
from pathlib import Path

root = Path("/content/tiny-imagenet-200")
val_dir = root/"val"
img_dir = val_dir/"images"
ann = val_dir/"val_annotations.txt"

with ann.open("r") as f:
    for line in f:
        img, cls = line.strip().split("\t")[:2]
        (val_dir/cls).mkdir(parents=True, exist_ok=True)
        src = img_dir/img
        dst = val_dir/cls/img
        if src.exists():
            shutil.move(str(src), str(dst))

if img_dir.exists():
    shutil.rmtree(img_dir)

print("✅ Tiny-ImageNet val reorganized into class subfolders.")

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')
✅ Tiny-ImageNet val reorganized into class subfolders.


In [ ]:
!find /content/tiny-imagenet-200/val -maxdepth 1 -type d | head

/content/tiny-imagenet-200/val
/content/tiny-imagenet-200/val/n03891332
/content/tiny-imagenet-200/val/n04275548
/content/tiny-imagenet-200/val/n07695742
/content/tiny-imagenet-200/val/n01950731
/content/tiny-imagenet-200/val/n01770393
/content/tiny-imagenet-200/val/n03393912
/content/tiny-imagenet-200/val/n03179701
/content/tiny-imagenet-200/val/n09256479
/content/tiny-imagenet-200/val/n04070727


In [ ]:
ls -lah /content/tiny-imagenet-200 | head

total 2.6M
drwxrwxr-x   5 root root 4.0K Jan 25 13:15 ./
drwxr-xr-x   1 root root 4.0K Jan 25 13:14 ../
drwxrwxr-x   3 root root 4.0K Dec 12  2014 test/
drwxrwxr-x 202 root root 4.0K Dec 12  2014 train/
drwxrwxr-x 202 root root 4.0K Jan 25 13:15 val/
-rw-rw-r--   1 root root 2.0K Feb  9  2015 wnids.txt
-rw-------   1 root root 2.6M Feb  9  2015 words.txt


Handle timm incompatibilities. Although we can instantiate the model directly, the training script uses timm.create_model(), which injects metadata arguments such as pretrained_cfg and cache_dir.
The original DeiT constructors do not support these arguments, so we remove them
YOUR NOTEBOOK CALL
    |
    v
deit_tiny_patch16_224()          ✅ works (no kwargs)

TRAINING PIPELINE
    |
    v
timm.create_model()
    |
    v
deit_tiny_patch16_224(**kwargs)  ❌ injects extra keys


Patch /content/deit/augment.py (safe compatibility fix)

In [ ]:
%cd /content/deit
!python - << 'PY'
from pathlib import Path
p = Path("augment.py")
txt = p.read_text()

old = "from timm.data.transforms import _pil_interp, RandomResizedCropAndInterpolation, ToNumpy, ToTensor"
if old in txt:
    txt = txt.replace(
        old,
        "from timm.data.transforms import RandomResizedCropAndInterpolation, ToNumpy, ToTensor\n"
        "try:\n"
        "    from timm.data.transforms import _pil_interp  # older timm\n"
        "except Exception:\n"
        "    _pil_interp = None  # newer timm doesn't expose this\n"
    )
    p.write_text(txt)
    print("✅ Patched augment.py for timm compatibility.")
else:
    print("ℹ️ Expected import line not found; augment.py may already be patched or different.")

/content/deit
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
ℹ️ Expected import line not found; augment.py may already be patched or different.


In [ ]:
%cd /content/deit
!sed -n '1,200p' models.py

/content/deit
# Copyright (c) 2015-present, Facebook, Inc.
# All rights reserved.
import torch
import torch.nn as nn
from functools import partial

from timm.models.vision_transformer import VisionTransformer, _cfg
from timm.models.registry import register_model
from timm.models.layers import trunc_normal_


__all__ = [
    'deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_base_patch16_224',
    'deit_tiny_distilled_patch16_224', 'deit_small_distilled_patch16_224',
    'deit_base_distilled_patch16_224', 'deit_base_patch16_384',
    'deit_base_distilled_patch16_384',
]


class DistilledVisionTransformer(VisionTransformer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dist_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        num_patches = self.patch_embed.num_patches
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 2, self.embed_dim))
        self.head_dist = nn.Linear(self.embed_dim, self.num_classes)

In [ ]:
%cd /content/deit
!rm -f multiteacher_loss.py
!ls -l multiteacher_loss.py || echo "✅ old file removed"

In [ ]:
%cd /content/deit

from pathlib import Path

code = r'''
from __future__ import annotations
from typing import Dict, List, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm


def normalize_lambdas(lmb: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    if lmb.dim() == 1:
        return lmb / lmb.sum().clamp_min(eps)
    return lmb / lmb.sum(dim=-1, keepdim=True).clamp_min(eps)


def fuse_logits(teacher_logits: Dict[str, torch.Tensor], teacher_order: List[str], lambdas: torch.Tensor) -> torch.Tensor:
    logits_list = [teacher_logits[k] for k in teacher_order]
    stacked = torch.stack(logits_list, dim=1)
    lambdas = normalize_lambdas(lambdas).to(stacked.device)
    if lambdas.dim() == 1:
        lambdas = lambdas.unsqueeze(0).expand(stacked.size(0), -1)
    return (stacked * lambdas.unsqueeze(-1)).sum(dim=1)


def kd_soft(student_logits: torch.Tensor, teacher_logits: torch.Tensor, T: float) -> torch.Tensor:
    T = float(T)
    p_t = F.softmax(teacher_logits / T, dim=-1)
    log_p_s = F.log_softmax(student_logits / T, dim=-1)
    return F.kl_div(log_p_s, p_t, reduction="batchmean") * (T * T)


def kd_hard(student_logits: torch.Tensor, teacher_logits: torch.Tensor) -> torch.Tensor:
    return F.cross_entropy(student_logits, teacher_logits.argmax(dim=-1))


class FrozenTeacherEnsemble(nn.Module):
    def __init__(self, teacher_names: List[str], device: torch.device):
        super().__init__()
        self.models = nn.ModuleDict({
            name: timm.create_model(name, pretrained=True, num_classes=1000).eval().to(device)
            for name in teacher_names
        })
        for m in self.models.values():
            for p in m.parameters():
                p.requires_grad_(False)
        self.teacher_order = list(self.models.keys())

    @torch.no_grad()
    def forward(self, x):
        return {k: m(x) for k, m in self.models.items()}


class TeacherLogitAdapter(nn.Module):
    def __init__(self, teacher_keys: List[str], student_num_classes: int):
        super().__init__()
        self.adapters = nn.ModuleDict({
            k: nn.Linear(1000, student_num_classes, bias=False) for k in teacher_keys
        })

    def forward(self, teacher_logits):
        return {k: self.adapters[k](v) for k, v in teacher_logits.items()}


class HDTSEConfidence(nn.Module):
    def __init__(self, temp=1.0):
        super().__init__()
        self.temp = temp

    @torch.no_grad()
    def forward(self, student_logits, teacher_logits, teacher_order, targets):
        stacked = torch.stack([teacher_logits[k] for k in teacher_order], dim=1)  # (B,T,C)
        probs = F.softmax(stacked / self.temp, dim=-1)  # (B,T,C)

        # Case 1: hard labels (B,)
        if targets.dim() == 1:
            idx = targets.to(dtype=torch.long, device=probs.device)
            conf = probs.gather(-1, idx[:, None, None]).squeeze(-1)  # (B,T)
            return normalize_lambdas(conf)

        # Case 2: soft labels from mixup/cutmix (B,C)
        # Confidence = expected probability under the soft label distribution
        tgt = targets.to(dtype=probs.dtype, device=probs.device)     # (B,C)
        conf = (probs * tgt[:, None, :]).sum(dim=-1)                 # (B,T)
        return normalize_lambdas(conf)


class MultiTeacherDistillationLoss(nn.Module):
    def __init__(
        self,
        base_criterion,
        student_num_classes,
        teacher_names,
        distillation_type="soft",
        alpha=0.5,
        tau=2.0,
        device=None,
        use_adapter=True,
    ):
        super().__init__()
        self.base_criterion = base_criterion
        self.distillation_type = distillation_type
        self.alpha = alpha
        self.tau = tau
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.teachers = FrozenTeacherEnsemble(teacher_names, self.device)
        self.adapter = TeacherLogitAdapter(self.teachers.teacher_order, student_num_classes).to(self.device) if use_adapter else None
        self.hdtse = HDTSEConfidence()

    def forward(self, inputs, outputs, targets):
        base_loss = self.base_criterion(outputs, targets)

        with torch.no_grad():
            t_logits = self.teachers(inputs)
        if self.adapter:
            t_logits = self.adapter(t_logits)

        order = self.teachers.teacher_order
        lambdas = self.hdtse(outputs, t_logits, order, targets)
        fused = fuse_logits(t_logits, order, lambdas)

        kd = kd_soft(outputs, fused, self.tau) if self.distillation_type == "soft" else kd_hard(outputs, fused)
        return (1 - self.alpha) * base_loss + self.alpha * kd
'''

path = Path("multiteacher_loss.py")
path.write_text(code)

print("File written:", path)
print("File size (bytes):", path.stat().st_size)

In [ ]:
%cd /content/deit

!python - <<'PY'
from pathlib import Path
import re

p = Path("main.py")
txt = p.read_text()

# 1) Import our new loss
if "from multiteacher_loss import MultiTeacherDistillationLoss" not in txt:
    txt = txt.replace(
        "from losses import DistillationLoss",
        "from losses import DistillationLoss\nfrom multiteacher_loss import MultiTeacherDistillationLoss"
    )

# 2) Add CLI arg for multiple teachers
if "--teacher-models" not in txt:
    txt = txt.replace(
        "parser.add_argument('--teacher-path', type=str, default='')",
        "parser.add_argument('--teacher-path', type=str, default='')\n"
        "    parser.add_argument('--teacher-models', type=str, default='',\n"
        "                        help='Comma-separated timm model names for multi-teacher distillation')"
    )

# 3) Replace criterion construction block
# We search for the block that instantiates DistillationLoss and replace it with a multi-teacher switch.
pattern = r"(criterion\s*=\s*DistillationLoss\([\s\S]*?\)\n)"
m = re.search(pattern, txt)
if m:
    block = m.group(1)
    repl = (
        "    # --- Multi-teacher distillation (Layer 2) ---\n"
        "    if args.teacher_models:\n"
        "        teacher_names = [t.strip() for t in args.teacher_models.split(',') if t.strip()]\n"
        "        criterion = MultiTeacherDistillationLoss(\n"
        "            base_criterion=criterion,\n"
        "            student_num_classes=args.nb_classes,\n"
        "            teacher_names=teacher_names,\n"
        "            distillation_type=args.distillation_type,\n"
        "            alpha=args.distillation_alpha,\n"
        "            tau=args.distillation_tau,\n"
        "            device=device,\n"
        "            use_adapter=True,\n"
        "        )\n"
        "    else:\n"
        + block
    )
    txt = txt[:m.start()] + repl + txt[m.end():]
else:
    print("WARNING: Could not auto-find DistillationLoss block. You'll need to patch main.py manually by searching for 'DistillationLoss('.")

p.write_text(txt)
print("✅ Patched main.py for multi-teacher loss.")

In [ ]:
from pathlib import Path
import re
import shutil
from datetime import datetime

MAIN = Path("/content/deit/main.py")

# 1) Backup
backup = MAIN.with_name(f"main.py.bak_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
shutil.copy2(MAIN, backup)
print("✅ Backup created:", backup)

txt = MAIN.read_text()

# 2) Canonical distillation block (single-teacher OR multi-teacher, no duplicates)
replacement_block = r"""
    teacher_model = None
    if args.distillation_type != 'none':
        assert (args.teacher_path or args.teacher_models), 'need to specify teacher-path OR teacher-models when using distillation'

        if args.teacher_models:
            teacher_names = [t.strip() for t in args.teacher_models.split(',') if t.strip()]
            teacher_model = None
            criterion = MultiTeacherDistillationLoss(
                base_criterion=criterion,
                student_num_classes=model_without_ddp.head.out_features,
                teacher_names=teacher_names,
                distillation_type=args.distillation_type,
                alpha=args.distillation_alpha,
                tau=args.distillation_tau,
                device=device,
                use_adapter=True,
            )
        else:
            # Single-teacher DeiT distillation path (expects a checkpoint at --teacher-path)
            print(f"Creating teacher model: {args.teacher_model}")
            teacher_model = create_model(
                args.teacher_model,
                pretrained=False,
                num_classes=args.nb_classes,
                global_pool='avg',
            )
            if args.teacher_path.startswith('https'):
                checkpoint = torch.hub.load_state_dict_from_url(
                    args.teacher_path, map_location='cpu', check_hash=True)
            else:
                checkpoint = torch.load(args.teacher_path, map_location='cpu')
            teacher_model.load_state_dict(checkpoint['model'])
            teacher_model.to(device)
            teacher_model.eval()
            for p in teacher_model.parameters():
                p.requires_grad_(False)

            criterion = DistillationLoss(
                criterion,
                teacher_model,
                args.distillation_type,
                args.distillation_alpha,
                args.distillation_tau,
            )
"""

# 3) Replace the entire problematic region:
#    from: "teacher_model = None"
#    to:   just before "output_dir = Path(args.output_dir)"
pattern = re.compile(
    r"\n\s*teacher_model\s*=\s*None\s*\n.*?\n\s*output_dir\s*=\s*Path\(args\.output_dir\)\s*\n",
    re.DOTALL
)

new_txt, n = pattern.subn("\n" + replacement_block + "\n\n    output_dir = Path(args.output_dir)\n", txt)

if n != 1:
    raise RuntimeError(f"Expected to replace exactly 1 distillation block, but replaced {n}. "
                       f"Open {backup} and inspect if needed.")

MAIN.write_text(new_txt)
print("✅ main.py patched successfully (distillation block rewritten cleanly).")

# 4) Quick sanity check: ensure there are NO duplicate 'Layer 2' blocks left
print("Occurrences of 'Multi-teacher distillation (Layer 2)' after patch:",
      new_txt.count("Multi-teacher distillation (Layer 2)"))
print("Occurrences of 'if args.teacher_models' after patch:",
      new_txt.count("if args.teacher_models"))

Before constructing the model, remove those keys from kwargs

In [ ]:
from pathlib import Path

p = Path("/content/deit/models.py")
lines = p.read_text().splitlines()

out = []
for line in lines:
    out.append(line)
    if line.strip().startswith("def deit_") and "**kwargs" in line:
        out.append("    # Drop timm-injected kwargs not supported by DeiT")
        out.append("    kwargs.pop('pretrained_cfg', None)")
        out.append("    kwargs.pop('pretrained_cfg_overlay', None)")
        out.append("    kwargs.pop('pretrained_cfg_priority', None)")

p.write_text("\n".join(out) + "\n")
print("✅ models.py patched to drop pretrained_cfg kwargs")


✅ models.py patched to drop pretrained_cfg kwargs


Verify

In [ ]:
!grep -n "pretrained_cfg" /content/deit/models.py

65:    kwargs.pop('pretrained_cfg', None)
66:    kwargs.pop('pretrained_cfg_overlay', None)
67:    kwargs.pop('pretrained_cfg_priority', None)
73:    kwargs.pop('pretrained_cfg', None)
74:    kwargs.pop('pretrained_cfg_overlay', None)
75:    kwargs.pop('pretrained_cfg_priority', None)
85:    kwargs.pop('pretrained_cfg', None)
86:    kwargs.pop('pretrained_cfg_overlay', None)
87:    kwargs.pop('pretrained_cfg_priority', None)
101:    kwargs.pop('pretrained_cfg', None)
102:    kwargs.pop('pretrained_cfg_overlay', None)
103:    kwargs.pop('pretrained_cfg_priority', None)
120:    kwargs.pop('pretrained_cfg', None)
121:    kwargs.pop('pretrained_cfg_overlay', None)
122:    kwargs.pop('pretrained_cfg_priority', None)
128:    kwargs.pop('pretrained_cfg', None)
129:    kwargs.pop('pretrained_cfg_overlay', None)
130:    kwargs.pop('pretrained_cfg_priority', None)
140:    kwargs.pop('pretrained_cfg', None)
141:    kwargs.pop('pretrained_cfg_overlay', None)
142:    kwargs.pop('pretrained_cfg_prio

Fix: Patch /content/deit/models.py to drop pretrained_cfg=...

Patch models.py to also drop cache_dir (and friends)

In [ ]:
from pathlib import Path

p = Path("/content/deit/models.py")
lines = p.read_text().splitlines()

# Keys that timm may inject but DeiT constructors don't accept
DROP_KEYS = [
    "cache_dir",
    "hf_hub_id",
    "hf_hub_filename",
    "hf_hub_revision",
]

out = []
for line in lines:
    out.append(line)
    # Right after the comment line we previously inserted, add more pops once per function
    if line.strip() == "# Drop timm-injected kwargs not supported by DeiT":
        for k in DROP_KEYS:
            out.append(f"    kwargs.pop('{k}', None)")

p.write_text("\n".join(out) + "\n")
print("✅ Patched models.py to drop cache_dir/hf_hub* kwargs")


✅ Patched models.py to drop cache_dir/hf_hub* kwargs


Verify

In [ ]:
!grep -n "cache_dir" /content/deit/models.py

65:    kwargs.pop('cache_dir', None)
73:    kwargs.pop('cache_dir', None)
77:    kwargs.pop('cache_dir', None)
85:    kwargs.pop('cache_dir', None)
89:    kwargs.pop('cache_dir', None)
93:    kwargs.pop('cache_dir', None)
101:    kwargs.pop('cache_dir', None)
105:    kwargs.pop('cache_dir', None)
109:    kwargs.pop('cache_dir', None)
113:    kwargs.pop('cache_dir', None)
136:    kwargs.pop('cache_dir', None)
144:    kwargs.pop('cache_dir', None)
148:    kwargs.pop('cache_dir', None)
156:    kwargs.pop('cache_dir', None)
160:    kwargs.pop('cache_dir', None)
164:    kwargs.pop('cache_dir', None)
172:    kwargs.pop('cache_dir', None)
176:    kwargs.pop('cache_dir', None)
180:    kwargs.pop('cache_dir', None)
184:    kwargs.pop('cache_dir', None)
207:    kwargs.pop('cache_dir', None)
215:    kwargs.pop('cache_dir', None)
219:    kwargs.pop('cache_dir', None)
227:    kwargs.pop('cache_dir', None)
231:    kwargs.pop('cache_dir', None)
235:    kwargs.pop('cache_dir', None)
243:    kwargs.pop

In [ ]:
# %cd /content/deit
# !python main.py \
#   --model deit_tiny_patch16_224 \
#   --data-path /content/tiny-imagenet-200 \
#   --pretrained \
#   --epochs 1 \
#   --batch-size 64 \
#   --num_workers 2 \
#   --output_dir /content/deit_runs/smoke_test
# %cd /content/deit
# !python main.py \
#   --model deit_tiny_patch16_224 \
#   --data-path /content/tiny-imagenet-200 \
#   --epochs 1 \
#   --batch-size 128 \
#   --num_workers 4 \
#   --input-size 224 \
#   --opt adamw \
#   --lr 5e-4 \
#   --weight-decay 0.05 \
#   --sched cosine \
#   --aa rand-m9-mstd0.5 \
#   --reprob 0.25 \
#   --remode pixel \
#   --recount 1 \
#   --output_dir /content/deit_runs/tiny_imagenet
### correct one
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 3e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.1 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.1 \
#  --output_dir /content/deit_runs/tiny_imagenet_5ep
%cd /content/deit
!python main.py \
 --model deit_tiny_patch16_224 \
 --data-path /content/tiny-imagenet-200 \
 --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
 --epochs 10 \
 --batch-size 128 \
 --num_workers 4 \
 --input-size 224 \
 --opt adamw \
 --lr 2.5e-4 \
 --weight-decay 0.05 \
 --sched cosine \
 --warmup-epochs 2 \
 --smoothing 0.1 \
 --aa rand-m6-mstd0.5 \
 --reprob 0.1 \
 --drop-path 0.05 \
 --mixup 0.2 \
 --cutmix 0.0 \
 --mixup-prob 0.5 \
 --distillation-type soft \
 --distillation-alpha 0.1 \
 --distillation-tau 2.0 \
 --output_dir /content/deit_runs/tiny_imagenet \
 --teacher-models "tf_efficientnet_b0,tf_efficientnet_b1,mobilenetv3_large_100"
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 2.5e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.1 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.1 \
#  --distillation-type hard \
# --teacher-model regnety_160 \
# --teacher-path https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth \
#  --output_dir /content/deit_runs/tiny_imagenet_10ep
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_distilled_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 7e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.0 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.0 \
#  --distillation-type hard \
#  --distillation-alpha 0.7 \
#  --teacher-model regnety_160 \
#  --teacher-path https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth \
#  --output_dir /content/deit_runs/deit_tiny_distilled_10ep





/content/deit
Not using distributed mode
Namespace(batch_size=128, epochs=10, bce_loss=False, unscale_lr=False, model='deit_tiny_patch16_224', input_size=224, drop=0.0, drop_path=0.05, model_ema=True, model_ema_decay=0.99996, model_ema_force_cpu=False, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=None, momentum=0.9, weight_decay=0.05, sched='cosine', lr=0.00025, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, warmup_lr=1e-06, min_lr=1e-05, decay_epochs=30, warmup_epochs=2, cooldown_epochs=10, patience_epochs=10, decay_rate=0.1, color_jitter=0.3, aa='rand-m6-mstd0.5', smoothing=0.1, train_interpolation='bicubic', repeated_aug=True, train_mode=True, ThreeAugment=False, src=False, reprob=0.1, remode='pixel', recount=1, resplit=False, mixup=0.2, cutmix=0.0, cutmix_minmax=None, mixup_prob=0.5, mixup_switch_prob=0.5, mixup_mode='batch', teacher_model='regnety_160', teacher_path='', distillation_type='none', distillation_alpha=0.5, distillation_tau=1.0, cosub=False, finetune='htt

# **Layer 2: Base Environment — Teacher Models & Multi-Teacher Adaptations**

Layer 2 extends the baseline DeiT environment to support knowledge distillation from one or more teacher models. This layer is additive: it does not modify the baseline DeiT training loop unless explicitly stated.
It includes
1. Teacher Model Support (Single & Multiple)
2. Teacher Registry / Configuration
3. Multi-Teacher Fusion Mechanism (Adaptation Layer)
4. Distillation Loss Integration